In [246]:
import numpy as np

In [247]:
class ScratchKMeans():
    """
    K-means scratch implementation
    Parameters
    ----------
    n_clusters : int
      Number of clusters
    n_init : int
      How many times to change the initial value of the center point for calculation
    max_iter : int
      Maximum number of iterations in one calculation
    tol : float
      Margin of error between the center point and the center of gravity, which is the reference for ending the iteration
    verbose : bool
      True to output the learning process
    """

    def __init__(self, n_clusters, n_init, max_iter, tol, verbose=False):
        # Record hyperparameters as attributes
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.tol = tol
        self.verbose = verbose

    def _init_mu_k(self, X):
        rows = np.random.choice(X.shape[0], size=self.n_clusters, replace=False)
        return X[rows, :]

    def _compute_SSE(self, X, mu, r):
        '''
        Calculate SSE
        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
          Features of training data
        r : shape (n_samples, self.n_clusters)
          cluster assignment
        mu : shape (self.n_clusters, n_features)
          center points
        Return
        ----------
        SSE: shape (n_features, )
          SSE
        '''
        SSE = 0.0
        for k in range(0, self.n_clusters):
            SSE += r[:, k] @ (X - mu[k, :])**2
        return SSE
    
    def _allocate_r(self, X, mu):
        """
        Allocate data points X to the nearest center point.
        """
        distance_matrix = np.zeros((X.shape[0], n_clusters))
        for k in range(0, self.n_clusters):
            distance_matrix[:, k] = np.linalg.norm(X - mu[k, :], axis=1)
        r = np.zeros((X.shape[0], n_clusters))
        r[np.arange(len(r)), np.argmin(distance_matrix, axis=1)] = 1
        return r

    def _move_mu(self, X, r):
        """
        Moves mu to the mean (center of gravity)
        """
        mu = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(0, self.n_clusters):
            mu[k, :] = X[r[:, k] == 1].mean(axis=0)
        return mu

    def _learning(self, X, init_mu):
        mu = init_mu
        r = self._allocate_r(X, mu)
        prev_mu = mu
        i = 0
        while (i < self.max_iter):
            mu = self._move_mu(X, r)
            r = self._allocate_r(X, mu)
            if (abs(np.sum(mu - prev_mu)) <= self.tol):
                break
            prev_mu = mu
            i+=1
        return mu, r

    def _find_best_learnings(self, X):
        SSE_list = []
        mu_list = []
        r_list = []
        for i in range(0, self.n_init):
            mu, r = self._find_best_learnings(X, self._init_mu_k(X))
            SSE_list.append(self._compute_SSE(X, mu, r))
            mu_list.append(mu)
            r_list.append(r)
            if verbose:
                print("iter: {}   SSE: {}".format(i, SSE_list[-1]))
        min_idx = np.argmin(np.array(SSE_list))
        return SSE_list[min_idx], mu_list[min_idx], r_list[min_idx]

    def fit(self, X):
        """
        Calculate clustering by K-means
        Parameters
        ----------
        X : The following forms of ndarray, shape (n_samples, n_features)
            Features of training data
        """
        if self.verbose:
            print("learning...")
            self.SSE, self.mu, self.r = self._find_best_learnings(X)
        else:
            self.SSE, self.mu, self.r = self._find_best_learnings(X)

    def predict(self, X):
        """
        Calculate which cluster the input data belongs to
        """
        return self._allocate_r(X, self.mu) 